# Gold Layer – Development Notebook (Sprint 1)

## Purpose
This notebook contains the **Gold transformation logic under development** for:

- `gold_dev_seller_daily_perf`

Gold Dev is used to iterate quickly and validate metrics before promotion.

---

## Sprint 1 Context (Parallel Development Phase)

During early Sprint 1, Silver was still under parallel development and not yet finalized.
Therefore, this notebook was originally allowed to read from:

- `sl_stub_*` (stable snapshot for parallel dev), or
- `sl_dev_*` (development tables)

This enabled multiple workstreams (Silver + Gold + BI) to progress concurrently without blocking.

---

## Post-Silver Promotion Rule (Current)

Silver `sl_*` is now **promoted and locked**.

For promotion-ready Gold outputs, this notebook must read from **locked Silver tables only**:

- `sl_orders`
- `sl_order_items`
- `sl_order_reviews`
- `sl_sellers`
- (any other required `sl_*` tables)

IMPORTANT Do not promote Gold if it is still reading from `sl_stub_*` or `sl_dev_*`.

---

## Output Tables

This notebook writes to **Gold development tables** only:

- `gold_dev_seller_daily_perf`

Promotion to shared Gold (`gold_*`) is controlled by the PO after validation passes.

---

## Promotion Gate

Promotion is allowed only when:

1) Gold Dev reads from `sl_*` (locked Silver source of truth)
2) Gold validation notebook passes (grain, baseline totals, delivery/review sanity)
3) BI contract is satisfied (columns and grain)


In [1]:
CREATE OR REPLACE TABLE gold_dev_seller_daily_perf AS

-- Step 1: Aggregate order items to order level (one row per order)
WITH order_items_agg AS (
    SELECT 
        oi.order_id,
        oi.seller_id,
        COUNT(*) AS item_count,
        SUM(COALESCE(oi.price, 0)) AS order_revenue,
        SUM(COALESCE(oi.price, 0) + COALESCE(oi.freight_value, 0)) AS order_gmv
    FROM sl_order_items oi
    GROUP BY oi.order_id, oi.seller_id
),

-- Step 2: Calculate delivery facts at order level (one row per order)
order_delivery_facts AS (
    SELECT 
        o.order_id,
        CAST(o.order_purchase_timestamp AS DATE) AS order_date,
        
        -- Delivery duration (only for delivered orders)
        CASE 
            WHEN o.order_delivered_customer_date IS NOT NULL 
            THEN DATEDIFF(DAY, o.order_purchase_timestamp, o.order_delivered_customer_date)
        END AS delivery_days,
        
        -- Is this order late? (1 = late, 0 = on time or not delivered)
        CASE 
            WHEN o.order_delivered_customer_date IS NOT NULL 
             AND o.order_estimated_delivery_date IS NOT NULL
             AND o.order_delivered_customer_date > o.order_estimated_delivery_date 
            THEN 1 
            ELSE 0 
        END AS is_late,
        
        -- Flags for counting
        CASE WHEN o.order_delivered_customer_date IS NOT NULL THEN 1 ELSE 0 END AS is_delivered,
        CASE 
            WHEN o.order_delivered_customer_date IS NOT NULL 
             AND o.order_estimated_delivery_date IS NOT NULL
             AND o.order_delivered_customer_date <= o.order_estimated_delivery_date 
            THEN 1 ELSE 0 
        END AS is_on_time,
        
        -- For sum of delivery duration
        CASE 
            WHEN o.order_delivered_customer_date IS NOT NULL 
            THEN DATEDIFF(DAY, o.order_purchase_timestamp, o.order_delivered_customer_date)
            ELSE 0
        END AS delivery_duration_for_sum,
        
        -- For counting orders with valid duration
        CASE 
            WHEN o.order_delivered_customer_date IS NOT NULL 
             AND o.order_purchase_timestamp IS NOT NULL 
            THEN 1 ELSE 0 
        END AS has_valid_duration
        
    FROM sl_orders o
),

-- Step 3: Aggregate reviews to order level (one row per order)
order_reviews_agg AS (
    SELECT 
        r.order_id,
        AVG(r.review_score) AS order_avg_review_score
    FROM sl_order_reviews r
    GROUP BY r.order_id
),

-- Step 4: Join order-level facts together (still one row per order)
order_level_facts AS (
    SELECT 
        oia.order_id,
        oia.seller_id,
        odf.order_date,
        
        -- Item metrics (from order_items_agg)
        oia.item_count,
        oia.order_revenue,
        oia.order_gmv,
        
        -- Delivery metrics (from order_delivery_facts)
        odf.delivery_days,
        odf.is_late,
        odf.is_delivered,
        odf.is_on_time,
        odf.delivery_duration_for_sum,
        odf.has_valid_duration,
        
        -- Review metrics (from order_reviews_agg)
        ora.order_avg_review_score
        
    FROM order_items_agg oia
    INNER JOIN order_delivery_facts odf ON oia.order_id = odf.order_id
    LEFT JOIN order_reviews_agg ora ON oia.order_id = ora.order_id
),

-- Step 5: Join seller dimensions
order_with_seller_dims AS (
    SELECT 
        olf.*,
        s.seller_city,
        s.seller_state
    FROM order_level_facts olf
    LEFT JOIN sl_sellers s ON olf.seller_id = s.seller_id
)

-- Step 6: Final aggregation to seller × day grain
SELECT 
    -- Grain: seller × day
    order_date,
    seller_id,
    seller_city,
    seller_state,
    
    -- Core BI metrics (existing columns)
    COUNT(DISTINCT order_id) AS total_orders,
    SUM(item_count) AS total_items,
    SUM(order_revenue) AS total_revenue,
    SUM(order_gmv) AS total_gmv_incl_freight,
    
    -- Delivery metrics (existing)
    AVG(delivery_days) AS avg_delivery_days,
    
    -- Flag: 1 if any late order exists for the seller-day
    MAX(is_late) AS has_late_delivery,  
    
    -- Review metric (existing)
    AVG(order_avg_review_score) AS avg_review_score,
    
    -- Sprint 1 NEW delivery KPI columns
    SUM(is_delivered) AS delivered_orders,
    
    -- NOTE: includes early deliveries (delivered <= estimated)
    SUM(is_on_time) AS on_time_delivered_orders,    
    SUM(CASE WHEN is_delivered = 1 AND is_on_time = 0 THEN 1 ELSE 0 END) AS late_delivered_orders,
    SUM(delivery_duration_for_sum) AS sum_delivery_duration_days,
    SUM(has_valid_duration) AS delivered_orders_with_duration

FROM order_with_seller_dims

GROUP BY 
    order_date,
    seller_id,
    seller_city,
    seller_state

StatementMeta(, 12b9427b-421a-4f1e-9f75-a64e00959425, 2, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [2]:
-- Check table exists and basic stats
SELECT 
    COUNT(*) as total_rows,
    COUNT(DISTINCT seller_id) as unique_sellers,
    COUNT(DISTINCT order_date) as unique_dates,
    MIN(order_date) as earliest_date,
    MAX(order_date) as latest_date
FROM gold_dev_seller_daily_perf

StatementMeta(, 12b9427b-421a-4f1e-9f75-a64e00959425, 3, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 5 fields>

In [3]:
-- This MUST return 0 rows (no duplicates at seller-day grain)
SELECT 
    seller_id, 
    order_date, 
    COUNT(*) as duplicate_count
FROM gold_dev_seller_daily_perf
GROUP BY seller_id, order_date
HAVING COUNT(*) > 1
ORDER BY duplicate_count DESC

StatementMeta(, 12b9427b-421a-4f1e-9f75-a64e00959425, 4, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 3 fields>

In [4]:
-- Check if numbers make logical sense
SELECT 
    COUNT(*) as total_rows,
    
    -- These should ALL be 0 (logical errors)
    SUM(CASE WHEN total_items < total_orders THEN 1 ELSE 0 END) as items_less_than_orders_BAD,
    SUM(CASE WHEN total_gmv_incl_freight < total_revenue THEN 1 ELSE 0 END) as gmv_less_than_revenue_BAD,
    SUM(CASE WHEN delivered_orders > total_orders THEN 1 ELSE 0 END) as delivered_more_than_total_BAD,
    SUM(CASE WHEN on_time_delivered_orders > delivered_orders THEN 1 ELSE 0 END) as ontime_more_than_delivered_BAD,
    SUM(CASE WHEN late_delivered_orders > delivered_orders THEN 1 ELSE 0 END) as late_more_than_delivered_BAD,
    
    -- These are acceptable (nulls are okay)
    SUM(CASE WHEN avg_delivery_days IS NULL THEN 1 ELSE 0 END) as null_delivery_days_OK,
    SUM(CASE WHEN avg_review_score IS NULL THEN 1 ELSE 0 END) as null_review_scores_OK,
    SUM(CASE WHEN seller_city IS NULL THEN 1 ELSE 0 END) as null_cities_CHECK,
    SUM(CASE WHEN seller_state IS NULL THEN 1 ELSE 0 END) as null_states_CHECK
    
FROM gold_dev_seller_daily_perf

StatementMeta(, 12b9427b-421a-4f1e-9f75-a64e00959425, 5, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 10 fields>

In [5]:
-- Check if multi-item orders are counted correctly (no fanout)
WITH multi_item_orders AS (
    SELECT 
        oi.order_id,
        oi.seller_id,
        COUNT(*) as item_count,
        CAST(o.order_purchase_timestamp AS DATE) as order_date
    FROM sl_order_items oi
    INNER JOIN sl_orders o ON oi.order_id = o.order_id
    GROUP BY oi.order_id, oi.seller_id, CAST(o.order_purchase_timestamp AS DATE)
    HAVING COUNT(*) > 1
    LIMIT 1
)
SELECT 
    'Silver Source' as source,
    mio.order_id,
    mio.seller_id,
    mio.order_date,
    mio.item_count as items,
    1 as should_be_counted_as_orders
FROM multi_item_orders mio

UNION ALL

SELECT 
    'Gold Table' as source,
    NULL as order_id,
    g.seller_id,
    g.order_date,
    g.total_items as items,
    g.total_orders as actual_order_count
FROM gold_dev_seller_daily_perf g
WHERE g.seller_id = (SELECT seller_id FROM multi_item_orders)
  AND g.order_date = (SELECT order_date FROM multi_item_orders)


StatementMeta(, 12b9427b-421a-4f1e-9f75-a64e00959425, 6, Finished, Available, Finished)

<Spark SQL result set with 2 rows and 6 fields>

In [6]:
-- Validate delivery KPIs add up correctly
SELECT 
    order_date,
    seller_id,
    total_orders,
    delivered_orders,
    on_time_delivered_orders,
    late_delivered_orders,
    (on_time_delivered_orders + late_delivered_orders) as sum_ontime_late,
    delivered_orders - (on_time_delivered_orders + late_delivered_orders) as difference,
    CASE 
        WHEN delivered_orders = (on_time_delivered_orders + late_delivered_orders) 
        THEN 'MATCH ✓' 
        WHEN ABS(delivered_orders - (on_time_delivered_orders + late_delivered_orders)) <= 1
        THEN 'CLOSE (missing estimates)'
        ELSE 'MISMATCH ❌' 
    END as validation_status
FROM gold_dev_seller_daily_perf
WHERE delivered_orders > 0
ORDER BY ABS(delivered_orders - (on_time_delivered_orders + late_delivered_orders)) DESC
LIMIT 20

StatementMeta(, 12b9427b-421a-4f1e-9f75-a64e00959425, 7, Finished, Available, Finished)

<Spark SQL result set with 20 rows and 9 fields>

In [8]:
-- Manual validation - what orders did this seller actually have?
SELECT 
    o.order_id,
    COUNT(*) as items_in_order,
    SUM(oi.price) as order_revenue,
    SUM(oi.price + oi.freight_value) as order_gmv,
    o.order_delivered_customer_date,
    o.order_estimated_delivery_date,
    CASE 
        WHEN o.order_delivered_customer_date > o.order_estimated_delivery_date 
        THEN 1 ELSE 0 
    END as is_late
FROM sl_order_items oi
INNER JOIN sl_orders o ON oi.order_id = o.order_id
WHERE oi.seller_id = '004c9cd9d87a3c30c522c48c4fc'  -- Same seller
  AND CAST(o.order_purchase_timestamp AS DATE) = '2018-03-27'  -- Same date
GROUP BY o.order_id, o.order_delivered_customer_date, o.order_estimated_delivery_date

StatementMeta(, 12b9427b-421a-4f1e-9f75-a64e00959425, 9, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 7 fields>

In [10]:
-- Look at sample data
SELECT *
FROM gold_dev_seller_daily_perf
ORDER BY order_date DESC, total_orders DESC
LIMIT 20

StatementMeta(, 12b9427b-421a-4f1e-9f75-a64e00959425, 11, Finished, Available, Finished)

<Spark SQL result set with 20 rows and 16 fields>